# Material Splaterial!!!
You need to have ImageMagick installed on your machine.  I use `subprocess.call` to invoke `magick` as a command-line application.

## Import Python Dependencies

In [10]:
import subprocess
import requests
import json
import re
import os

## Open Materials

In [3]:
material_packs = json.load(open('MaterialPacks.json', 'r'))

result_dict = {}
for pack_name, pack in material_packs.items():
    png_infos = {}
    for mtl_name, mtl in pack.items():
        asset_id = [
            requests.get(
                f'https://assetdelivery.roblox.com/v1/asset/?id={asset_id}'
            ).text
            for asset_id in mtl['ids']
        ]
        mtl_result = dict(re.findall(r'<([a-z]+)>([0-9]+)', ''.join(asset_id)))
        print(pack_name, mtl_name, mtl_result)
        png_infos[mtl_name] = mtl_result
    result_dict[pack_name] = png_infos

FStringPartTexturePacksJson2022 aluminum {'alphamode': '2', 'tiling': '1', 'color': '9466552117', 'normal': '9429416517', 'metalness': '9424786272', 'roughness': '9424786620', 'height': '9424787028', 'usage': '3'}
FStringPartTexturePacksJson2022 asphalt {'alphamode': '2', 'tiling': '1', 'color': '9930003046', 'normal': '9429452082', 'roughness': '9429450346', 'height': '9429450811', 'usage': '3'}
FStringPartTexturePacksJson2022 basalt {'alphamode': '2', 'tiling': '1', 'color': '9920482056', 'normal': '9438413509', 'roughness': '9438412457', 'height': '9438413060', 'usage': '3'}
FStringPartTexturePacksJson2022 brick {'alphamode': '2', 'tiling': '1', 'color': '9920482813', 'normal': '9438453869', 'roughness': '9438453413', 'height': '9438453693', 'usage': '3'}
FStringPartTexturePacksJson2022 cobblestone {'alphamode': '2', 'tiling': '1', 'color': '9919718991', 'normal': '9438453869', 'roughness': '9438457470', 'height': '9438459171', 'usage': '3'}
FStringPartTexturePacksJson2022 concrete 

## Save Assets (as `png` files)

In [7]:
BLOCKLIST = {
    'alphamode',
    'tiling',
    'usage',
}

png_infos = []
for pack_name, pack in result_dict.items():
    for mtl_name, mtl in pack.items():
        for asset_name, asset_id in mtl.items():
            if asset_name in BLOCKLIST:
                continue

            dds_data = requests.get(
                f'https://assetdelivery.roblox.com/v1/asset/?id={asset_id}'
            ).content

            orig_file_name = f'./images/{pack_name}/{mtl_name}/{asset_name}.png'
            os.makedirs(os.path.dirname(orig_file_name), exist_ok=True)

            png_infos.append((orig_file_name, dds_data, pack_name, mtl_name, asset_name, asset_id))
            with open(orig_file_name, 'wb') as f:
                f.write(dds_data)
        print('Loaded:', pack_name, mtl_name)

Loaded: FStringPartTexturePacksJson2022 aluminum
Loaded: FStringPartTexturePacksJson2022 asphalt
Loaded: FStringPartTexturePacksJson2022 basalt
Loaded: FStringPartTexturePacksJson2022 brick
Loaded: FStringPartTexturePacksJson2022 cobblestone
Loaded: FStringPartTexturePacksJson2022 concrete
Loaded: FStringPartTexturePacksJson2022 crackedlava
Loaded: FStringPartTexturePacksJson2022 diamondplate
Loaded: FStringPartTexturePacksJson2022 fabric
Loaded: FStringPartTexturePacksJson2022 glacier
Loaded: FStringPartTexturePacksJson2022 glass
Loaded: FStringPartTexturePacksJson2022 granite
Loaded: FStringPartTexturePacksJson2022 grass
Loaded: FStringPartTexturePacksJson2022 ground
Loaded: FStringPartTexturePacksJson2022 ice
Loaded: FStringPartTexturePacksJson2022 leafygrass
Loaded: FStringPartTexturePacksJson2022 limestone
Loaded: FStringPartTexturePacksJson2022 marble
Loaded: FStringPartTexturePacksJson2022 metal
Loaded: FStringPartTexturePacksJson2022 mud
Loaded: FStringPartTexturePacksJson2022 

In [11]:
for (orig_file_name, dds_data, pack_name, mtl_name, asset_name, asset_id) in png_infos:
    if asset_name == 'color':
        new_asset_names = ['diffuse']
    if asset_name == 'roughness':
        new_asset_names = ['reflection']
    if asset_name == 'normal':
        new_asset_names = ['normal', 'normaldetail']
    else:
        new_asset_names = [asset_name]

    for new_asset_name in new_asset_names:
        new_file_name = f'./textures/{pack_name}/{mtl_name}/{new_asset_name}.dds'
        os.makedirs(os.path.dirname(new_file_name), exist_ok=True)
        subprocess.call([
            'magick',
            orig_file_name,
            new_file_name
        ])